In [1]:
%load_ext sql
%sql sqlite:///

/Users/tarabalakrishnan/.local/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/tarabalakrishnan/.local/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


u'Connected: None@'

Problem Set 2
=============


### Instructions / Notes:

**_Read these carefully_**

* This problem set does **not** come with a dataset to load; instead, make your own instances of tables, either as solutions to the problems or for testing solutions to the problems.
* You are encouraged to create new IPython notebook cells to use for testing, debugging, exploring, etc. **Just make sure that your final answer for each question is in its own cell when you submit.**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging (i.e. running for too long): You can restart the SQL connection by restarting the entire python kernel.**
    * To restart the python kernel: Use the menu bar (Kernel >> Restart).
    * To restart the SQL connection: Re-execute the SQL connection cell at the top of the notebook
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* **See Piazza for submission instructions**
* _Have fun!_

Problem 1
---------

**_[20 points total]_**

For each part of this problem you will provide a _single_ SQL query to check whether a certain condition holds on a specific instance of a relation in the following way: **your query should return an empty result if and only if the condition holds on the instance.**  (If the condition _doesn't hold_, your query should return something non-empty, but it doesn't matter what this is).

Note our language here: the conditions that we specify cannot be proved to hold **in general** without knowing the externally-defined functional dependencies. This means that you are _checking whether they **could** hold in general for the relation, given any specific set of tuples_.

You may assume that there will be no `NULL` values in the tables, **and you may assume that the relations are _sets_ rather than multisets**, but otherwise your query should work for general instances.  We define the schemas of the tables used below for convenience, but in this problem you will need to construct your own test tables if you wish to use them to check your answers!

In [2]:
%%sql
DROP TABLE IF EXISTS R; CREATE TABLE R (A INT, B INT, C INT, D INT, E INT);
DROP TABLE IF EXISTS S; CREATE TABLE S (A INT, B INT, C INT);

Done.
Done.
Done.
Done.


[]

In [27]:
%%sql
DROP TABLE IF EXISTS R; 
CREATE TABLE R (A INT, B INT, C INT, D INT, E INT);
INSERT INTO R VALUES(0, 0, 0, 0, 0);

Done.
Done.
1 rows affected.


[]

### Part (a)

**_[5 points]_**

$\{A, B\} \rightarrow \{C\}$ and $\{C\} \rightarrow \{A, B\}$ hold for a relation $R(A,B,C,D,E)$.

In [29]:
%%sql
SELECT DISTINCT R1.A, R1.B, R1.C
FROM R as R1, R as R2
WHERE (R1.A = R2.A 
AND R1.B = R2.B 
AND R1.C <> R2.C) or 
(R1.C = R2.C and R1.B <> R2.B or R1.A <> R2.A);

Done.


A,B,C


### Part (b)

**_[5 points]_**

$\{A, B, C\}$ is a **superkey** for a relation $R(A,B,C,D,E)$.

In [ ]:
%%sql
SELECT *
FROM R r1, R r2
WHERE (r1.A = r2.A and r1.B = r2.B and r1.C = r2.C) and
      (r1.D <> r2.D or r1.E <> r2.E);

### Part (c)

**_[5 points]_**

$\{A\}$ and $\{B\}$ are each **keys** for a relation $S(A,B,C)$.

In [ ]:
%%sql
SELECT *
FROM S s1, S s2
WHERE (s1.A = s2.A and (s1.B <> s2.B or s1.C <> s2.C)) or
      (s1.B = s2.B and (s1.A <> s2.A or s1.C <> s2.C));

### Part (d)

**_[5 points]_**

A **multi-valued dependency (MVD)** is defined as follows: let $R$ be a schema i.e. a set of attributes, and consider two _sets of attributes_ $X\subseteq R$ and $Y\subseteq R$.  We say that a multi-valued dependency (MVD), written:

$X\twoheadrightarrow Y$

**holds on $R$** if whenever there are two tuples $t_1,t_2$ such that $t_1[X] = t_2[X]$, there also exists a third tuple $t_3$ such that:

* $t_3[X] = t_1[X] = t_2[X]$
* $t_3[Y] = t_1[Y]$
* $t_3[R \setminus Y] = t_2[R \setminus Y]$

Note that $R \setminus B$ is all the attributes in $R$ that are not in $B$, and that $t_3$ need not be distinct from $t_1$ or $t_2$.  Note especially that an MVD holds on an entire _relation_, meaning that any two tuples (in any order) in the relation should satisfy the above conditions if the MVD holds.  **See the end of the lecture 7 slides for more on MVDs!**


In this problem, write your query to check if the MVD $\{A\}\twoheadrightarrow \{C,E\}$ holds for a relation $R(A,B,C,D,E)$.

In [ ]:
%%sql
SELECT *
FROM R t1, R t2
WHERE t1.A = t2.A and
      NOT EXISTS (
          SELECT *
          FROM R t3
          WHERE t3.A = t1.A and
                t3.C = t1.C and
                t3.E = t1.E and
                t3.B = t2.B and
                t3.D = t2.D);

Problem 2
---------

Consider a relation $R(X, Y, Z)$ with some list of functional dependencies $f_1, f_2, \ldots, f_n$. Suppose that $K$ is a **superkey** for this relation, given these functional dependencies (recall that any superkey $K$ must be a subset of the attributes of $R$, which are $\{X, Y, Z\}$). In each part of this problem, we will examine what can happen if we add an additional functional dependency to our relation.

To answer **yes**, provide python code that assigns the variable ```answer``` to ```True``` and assigns ```explanation``` to be a python string which contains a (short!) explanation of why.  For example:

```python
answer = True
explanation = "Alex is correct because all keys are superkeys."
```

To answer **no**, provide python code that assigns the variable ```answer``` to ```False```.  You must also assign the variable ```K``` to be a set of attributes, the variable ```FDs``` (functional dependencies) to be a python list having elements that are _pairs_ of sets, and the variable ``new_FD`` (the new functional dependency to be added) to be a pair of sets, such that these three variables together produce a counter-example for the desired statement.  For example, a counter-example to the statement "if $K \rightarrow \{Z\}$ is false, then it will remain false when we add a new functional depencency" could look like: 

```python
answer = False
K = set("X")
FDs = [
    (set("Y"), set("Z")),
    (set(("Y", "Z")), set("X"))
]
new_FD = (set("X"), set("Z"))
```

### Part (a)

CS145 student Alex claims that if we add any new functional dependency $f_{n+1} = U \rightarrow V$ (where $U$ and $V$ are subsets of $\{X, Y, Z\}$) to our list of functional dependencies, then $K$ will still be a superkey for $R$ given $f_1, f_2, \ldots, f_{n+1}$.  Is Alex correct?  If yes, explain why.  If no, provide a counter-example.

In [ ]:
answer = True
explanation = \
"""
Consider the original set of functional dependencies $f_1,f_2,\ldots,f_n$.
The assumption that $K$ is a superkey implies that the original set of FDs implies the FD $K \rightarrow {A}$ for any attribute A.
Adding an additional functional dependency cannot cause any of the inferred conditions to become false, so all of the conditions for $K$ to be a superkey will still hold.
"""

### Part (b)

Consider again a relation $R(X, Y, Z)$ with some list of functional dependencies $f_1, f_2, \ldots, f_n$. Now suppose that $K$ is a **key** for this relation, given these functional dependencies.

CS145 student Bryan claims that if we add any new functional dependency $f_{n+1} = U \rightarrow V$ to our list of functional dependencies, then $K$ will still be a key for $R$ given $f_1, f_2, \ldots, f_{n+1}$.  Is Bryan correct?  If yes, explain why.  If no, provide a counter-example.

In [ ]:
answer = False
X = "X"
Y = "Y"
Z = "Z"
K = set((X, Y))
FDs = [(set((X, Y)), set(Z))]
new_FD = (set(X), set(Y))

### Part (c)

Consider now a more general relation $R(X_1, X_2, \ldots, X_m)$ with some list of functional dependencies $f_1, f_2, \ldots, f_n$. Suppose again that $K$ is a **key** for this relation, given these functional dependencies (recall that any superkey $K$ must be a subset of the attributes of $R$, which are $\{X_1, X_2, \ldots X_m\}$).

CS145 student Chris claims that if we add any new functional dependency $f_{n+1} = U \rightarrow V$ (where $U$ and $V$ are subsets of $\{X_1, X_2, \ldots, X_m\}$) to our list of functional dependencies, and the new functional dependency **does not include any of the attributes that are in the key $K$** (that is, $U \cap K = V \cap K = \emptyset$), then $K$ will still be a key for $R$ given $f_1, f_2, \ldots, f_{n+1}$.  Is Chris correct?  If yes, explain why.  If no, provide a counter-example.

In [ ]:
answer = False
X1 = "A"
X2 = "B"
X3 = "C"
X4 = "D"
K = set((X1, X2))
FDs = [(set((X1, X2)), set((X3, X4))),
       (set((X3, X4)), set(X2)),
       (set(X1), set(X3))]
new_FD = (set(X3), set(X4))

Problem 3
---------

Dan wants to find the keys of a relation based on an instance (the concrete rows stored in a database), rather than the functional dependencies (which are external constraints on the schema), but he isn't sure when this problem is possible. In this problem, you will study potential errors that can occur when using an instance instead of the functional dependencies.

Dan defines a set $K$ to be a **plausible key** for an instance `T` if $K$ could be a superkey for `T`, and no subset of $K$ could possibly be a superkey for `T`.  (Equivalently, $K$ is a plausible key for `T` if there is an set of functional dependencies that are consistent with `T` in which $K$ is a superkey, and for all subsets $U \subset K$, there are no sets of functional dependencies that are consistent with `T` in which $U$ is a superkey.)

Consider a relation $R(A, B, C, D)$ that satisfies the following set of functional dependencies:
\begin{align}
\{A, B\} &\rightarrow \{C\} \\
\{C\} &\rightarrow \{D\} \\
\{D\} &\rightarrow \{A, B\}
\end{align}

### Part (a)

**_[10 points]_**

Create an instance `T` of $R$ for which $\{A, B\}$ is a plausible key. If you believe that `T` cannot be created, provide it as an empty table.

Use a series of `INSERT` statements below to populate the table `T`.

In [ ]:
%%sql
DROP TABLE IF EXISTS T; 
CREATE TABLE T(A int, B int, C int, D int);

INSERT INTO T VALUES(0, 0, 0, 0);
INSERT INTO T VALUES(0, 1, 1, 1);
INSERT INTO T VALUES(1, 0, 2, 2);
INSERT INTO T VALUES(1, 1, 3, 3);
INSERT INTO T VALUES(1, 2, 4, 5);

### Part (b)

**_[10 points]_**

Create an instance `T` of $R$ for which $\{A\}$ is a plausible key. If you believe that `T` cannot be created, provide it as an empty table.

Use a series of `INSERT` statements below to populate the table `T`.

In [ ]:
%%sql
DROP TABLE IF EXISTS T; 
CREATE TABLE T(A int, B int, C int, D int);

INSERT INTO T VALUES(0, 0, 0, 0);

### Part (c)

**_[10 points]_**

Create an instance `T` of $R$ for which $\{A, B, C\}$ is a plausible key. If you believe that `T` cannot be created, provide it as an empty table.

Use a series of `INSERT` statements below to populate the table `T`.

In [ ]:
%%sql
DROP TABLE IF EXISTS T; 
CREATE TABLE T(A int, B int, C int, D int);

Problem 4
---------

### Part (a)

**_[10 points]_**

Consider a schema $R(A, B, C, D)$ which has functional dependencies
\begin{align}
    \{A, C\} &\rightarrow \{ B \} \\
    \{D \} &\rightarrow \{ C \}. \\
\end{align}
Create an instance of $R$ which is consistent with these functional dependencies, but not with any other functional dependencies (i.e. **all functional dependencies that are not inferred from these are violated**).

Use a series of `INSERT` statements below to populate the table `R`.

In [ ]:
%%sql
DROP TABLE IF EXISTS R; 
CREATE TABLE R (A int, B int, C int, D int);

INSERT INTO R VALUES(0, 0, 0, 0);

-- Violate {A, B, C} => {D}
INSERT INTO R VALUES(0, 0, 0, 1);

-- Nothing to violate for {A, B, D}
-- Nothing to violate for {A, C, D}
-- Violate {B, C, D} => {A}
INSERT INTO R VALUES(2, 0, 0, 0);

-- Violate {A, B} => {C, D}
INSERT INTO R VALUES(0, 0, 3, 3);

-- Violate {A, C} => {D}
-- (already violated for {A, B, C} => {D})

-- Nothing to violate for {A, D}
-- Violate {B, C} => {A, D}
INSERT INTO R VALUES(4, 0, 0, 4);

-- Violate {B, D} => {A}
-- (Already violated by {B, C, D} => {A})

-- Violate {C, D} => {A, B}
INSERT INTO R VALUES(5, 5, 0, 0);

-- Violate {A} => {B, C, D}
INSERT INTO R VALUES(0, 6, 6, 6);

-- Violate {B} => {A, C, D}
INSERT INTO R VALUES(7, 0, 7, 7);

-- Violate {C} => {A, B, D}
INSERT INTO R VALUES(8, 8, 0, 8);

-- Violate {D} => {A, B}
-- (Already violated by {C, D} => {A, B})

### Part (b)

**_[10 points]_**

Now consider a schema $S(A, B, C, D, E)$ which has an additional attribute $E$, but the same functional dependencies
\begin{align}
    \{A, C\} &\rightarrow \{ B \} \\
    \{D \} &\rightarrow \{ C \}. \\
\end{align}
Create an instance of $S$ which is consistent with these functional dependencies, but not with any other functional dependencies (i.e. **all functional dependencies that are not inferred from these are violated**).

Write one or more SQL statements that populate the table `S` using the table `R` you constructed in part (a).

In [ ]:
%%sql
DROP TABLE IF EXISTS S;
CREATE TABLE S (A int, B int, C int, D int, E int);

INSERT INTO S SELECT *, 0 FROM R;
INSERT INTO S SELECT *, 1 FROM R;